In [11]:
from dataget import data
import tfinterface as ti
import sonnet as snt
import tensorflow as tf

In [2]:
dataset = data("mnist").get()

In [3]:
traning_set = dataset.complete_set

In [55]:
class Discriminator(snt.AbstractModule):
    
    def _build(self, inputs):
        
        net = inputs["image"]; print(net)
        training = inputs["mode"] == tf.estimator.ModeKeys.TRAIN
        
        net = ti.layers.conv2d_batch_norm(net, 16, [5, 5], strides = 2, activation = tf.nn.elu, padding = "same", batch_norm = dict(training = training)); print(net)
        net = ti.layers.conv2d_batch_norm(net, 32, [3, 3], strides = 2, activation = tf.nn.elu, padding = "valid", batch_norm = dict(training = training)); print(net)
        net = ti.layers.conv2d_batch_norm(net, 64, [3, 3], strides = 2, activation = tf.nn.elu, padding = "valid", batch_norm = dict(training = training)); print(net)
        net = ti.layers.conv2d_batch_norm(net, 128, [2, 2], strides = 2, activation = tf.nn.elu, padding = "same", batch_norm = dict(training = training)); print(net)

        
        net = tf.contrib.layers.flatten(net); print(net)
        
        net = ti.layers.dense_batch_norm(net, 64, activation = tf.nn.elu, batch_norm = dict(training = training)); print(net)
        net = logits = tf.layers.dense(net, 1, activation = None); print(net)
#         net = tf.nn.sigmoid(net); print(net)
        
        return logits
    
    
class Generator(snt.AbstractModule):
    
    def _build(self, inputs):
        
        net = inputs["image"]; print(net)
        training = inputs["mode"] == tf.estimator.ModeKeys.TRAIN
        
        net = ti.layers.conv2d_batch_norm(net, 16, [5, 5], strides = 2, activation = tf.nn.elu, padding = "same", batch_norm = dict(training = training)); print(net)
        net = ti.layers.conv2d_batch_norm(net, 32, [3, 3], strides = 2, activation = tf.nn.elu, padding = "valid", batch_norm = dict(training = training)); print(net)
        net = ti.layers.conv2d_batch_norm(net, 64, [3, 3], strides = 2, activation = tf.nn.elu, padding = "valid", batch_norm = dict(training = training)); print(net)
        net = ti.layers.conv2d_batch_norm(net, 128, [2, 2], strides = 2, activation = tf.nn.elu, padding = "same", batch_norm = dict(training = training)); print(net)

        
        net = tf.contrib.layers.flatten(net); print(net)
        
        net = ti.layers.dense_batch_norm(net, 64, activation = tf.nn.elu, batch_norm = dict(training = training)); print(net)
        net = logits = tf.layers.dense(net, 1, activation = None); print(net)
#         net = tf.nn.sigmoid(net); print(net)
        
        return logits

In [56]:
inputs = dict(
    image = tf.layers.Input(shape=(28, 28, 1)),
    mode = tf.estimator.ModeKeys.TRAIN
)

d = Discriminator(name = "discriminator")
d(inputs)

AAAAAAAAA
Tensor("input_layer_5:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("discriminator_19/Conv2dBatchNorm/Elu:0", shape=(?, 14, 14, 16), dtype=float32)
Tensor("discriminator_19/Conv2dBatchNorm_1/Elu:0", shape=(?, 6, 6, 32), dtype=float32)
Tensor("discriminator_19/Conv2dBatchNorm_2/Elu:0", shape=(?, 2, 2, 64), dtype=float32)
Tensor("discriminator_19/Conv2dBatchNorm_3/Elu:0", shape=(?, 1, 1, 128), dtype=float32)
Tensor("discriminator_19/Flatten/flatten/Reshape:0", shape=(?, 128), dtype=float32)
Tensor("discriminator_19/DenseBatchNorm/Elu:0", shape=(?, 64), dtype=float32)
Tensor("discriminator_19/dense/BiasAdd:0", shape=(?, 1), dtype=float32)


<tf.Tensor 'discriminator_19/dense/BiasAdd:0' shape=(?, 1) dtype=float32>

Tensor("discriminator_1/conv2d/Elu:0", shape=(?, 12, 12, 16), dtype=float32)
Tensor("discriminator_1/conv2d_2/Elu:0", shape=(?, 5, 5, 32), dtype=float32)
Tensor("discriminator_1/conv2d_3/Elu:0", shape=(?, 2, 2, 64), dtype=float32)


<tf.Tensor 'discriminator_1/conv2d_3/Elu:0' shape=(?, 2, 2, 64) dtype=float32>